In [1]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import numpy as np
from yellowcard.coordinates import LocalGroupHalocentric

In [2]:
fiducial_ra = 10.68470833 * u.deg
fiducial_dec = 41.26875 * u.deg

# van der Marel et al. 2012

In [3]:
vdm_galcen_frame = coord.Galactocentric(
    galcen_distance=8.29*u.kpc, 
    galcen_v_sun=[11.1, 12.24 + 239.3, 7.25] * u.km/u.s, 
    z_sun=0*u.pc
)

In [4]:
vdm_galcen_frame_attrs = {k: getattr(vdm_galcen_frame, k) 
                          for k in vdm_galcen_frame.get_frame_attr_names()
                          if k != 'galcen_coord'}
vdm_galcen_frame_attrs

{'galcen_distance': <Quantity 8.29 kpc>,
 'galcen_v_sun': <CartesianDifferential (d_x, d_y, d_z) in km / s
     (11.1, 251.54, 7.25)>,
 'z_sun': <Quantity 0. pc>,
 'roll': <Quantity 0. deg>}

In [5]:
name = 'vdm2012'

# vw and vn + error
vW, vW_err = (-125.2, 30.8)* u.km/u.s # in kpc
vN, vN_err = (-73.8, 28.4)* u.km/u.s # in kpc
vW_term = (vW_err/vW)**2
vN_term = (vN_err/vN)**2

# distance to m31
D, D_err = (770, 40)* u.kpc # in kpc
D_term = (D_err/D)**2

# calculate equiv pms 
with u.set_enabled_equivalencies(u.dimensionless_angles()):
    mu_alpha_star = (- vW / D).to(u.microarcsecond/u.yr)
    mu_delta = (vN / D).to(u.microarcsecond/u.yr)

# equiv pm errors
mu_alpha_star_err = np.sqrt( mu_alpha_star**2 * (vW_term - D_term) ) 
mu_delta_err = np.sqrt( mu_delta**2 * (vN_term - D_term) ) 

tbl = {}
tbl['ra'] = fiducial_ra
tbl['dec'] = fiducial_dec

tbl['distance'] = D
tbl['distance_err'] = D_err

tbl['pm_ra_cosdec'] = mu_alpha_star
tbl['pm_ra_cosdec_err'] = mu_alpha_star_err

tbl['pm_dec'] = mu_delta
tbl['pm_dec_err'] = mu_delta_err

tbl['radial_velocity'] = -302 * u.km/u.s
tbl['radial_velocity_err'] = 1 * u.km/u.s

tbl['tperi'] = 13.75 * u.Gyr
tbl['tperi_err'] = 0.11 * u.Gyr

t = at.QTable({k: [] * tbl[k].unit for k in tbl if hasattr(tbl[k], 'unit')})
t.add_row(tbl)

t.meta['title'] = 'van der Marel et al. 2012'
t.meta['galcen_frame_attrs'] = vdm_galcen_frame_attrs

t.write(f'../datasets/{name}.ecsv', overwrite=True)

# van der Marel et al. 2012 — radial only

In [6]:
name = 'vdm2012-radial'

# distance to m31
D, D_err = [770, 40] * u.kpc

m31_c = coord.SkyCoord(
    ra=fiducial_ra,
    dec=fiducial_dec,
    distance=D
)
m31_lgh = m31_c.transform_to(vdm_galcen_frame).transform_to(LocalGroupHalocentric())
pos = m31_lgh.data.represent_as(coord.SphericalRepresentation)
v_rad_only = coord.SphericalDifferential(
    0*u.mas/u.yr, 0*u.mas/u.yr, -110*u.km/u.s)
lgh = LocalGroupHalocentric(pos.with_differentials(v_rad_only))
v_rad_only_icrs = lgh.transform_to(vdm_galcen_frame).transform_to(coord.ICRS())


tbl = {}
tbl['ra'] = fiducial_ra
tbl['dec'] = fiducial_dec

tbl['distance'] = D
tbl['distance_err'] = D_err

tbl['pm_ra_cosdec'] = v_rad_only_icrs.pm_ra_cosdec
tbl['pm_ra_cosdec_err'] = 1 * u.microarcsecond / u.yr

tbl['pm_dec'] = v_rad_only_icrs.pm_dec
tbl['pm_dec_err'] = 1 * u.microarcsecond / u.yr

tbl['radial_velocity'] = v_rad_only_icrs.radial_velocity
tbl['radial_velocity_err'] = 1 * u.km/u.s

tbl['tperi'] = 13.75 * u.Gyr
tbl['tperi_err'] = 0.11 * u.Gyr

t = at.QTable({k: [] * tbl[k].unit for k in tbl})
t.add_row(tbl)

t.meta['title'] = 'van der Marel et al. 2012 - radial only'

t.write(f'../datasets/{name}.ecsv', overwrite=True)

# Fiducial 2021 values

In [7]:
name = 'fiducial2021'

# distance to m31: Li et al. 2021
# https://ui.adsabs.harvard.edu/abs/2021arXiv210708029L/abstract
D, D_err = [761, 11] * u.kpc

tbl = {}
tbl['ra'] = fiducial_ra
tbl['dec'] = fiducial_dec

tbl['distance'] = D
tbl['distance_err'] = D_err

# Salomon et al. 2021
# https://arxiv.org/abs/2012.09204
tbl['pm_ra_cosdec'] = 48.98 * u.microarcsecond / u.yr
tbl['pm_ra_cosdec_err'] = 10.47 * u.microarcsecond / u.yr

tbl['pm_dec'] = -36.85 * u.microarcsecond / u.yr
tbl['pm_dec_err'] = 8.03 * u.microarcsecond / u.yr

tbl['radial_velocity'] = -302 * u.km/u.s
tbl['radial_velocity_err'] = 1 * u.km/u.s

tbl['tperi'] = 13.75 * u.Gyr
tbl['tperi_err'] = 0.11 * u.Gyr

t = at.QTable({k: [] * tbl[k].unit for k in tbl})
t.add_row(tbl)

t.meta['title'] = 'Fiducial 2021 data'

t.write(f'../datasets/{name}.ecsv', overwrite=True)